In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
testid = test['id']
train.head()

C:\Users\matt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,label,website,city,state,zip,country,registration_ip,banned_ip,ip_count,...,ip_cluster,email_cluster,mfa_required,mfa_completed,whitelabel_required,whitelabel_completed,payment_required,payment_completed,profile_completed,email_completed
0,0,1,http://tvtropes.org/,Manassas,VA,NaN,US,207.244.77.8,6,8,...,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,1,http://memebau.com,Tulsa,OK,NaN,US,104.168.149.15,5,5,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,http:/www./ShellsFor.Me,Ajman,AJ,NaN,AE,109.177.112.218,0,1,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,www.hjhj.com.mx,Mexico City,CMX,NaN,MX,201.137.247.62,4,4,...,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,4,0,jk.edu.br,Brasília,DF,NaN,BR,179.176.123.238,0,1,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
train.isnull().any()

id                          False
label                       False
website                      True
city                         True
state                        True
zip                          True
country                      True
registration_ip             False
banned_ip                   False
ip_count                    False
company                      True
created_at                  False
multifactor_country_code     True
is_authy_verified           False
banned_email                False
lead_source                  True
marketing_channel            True
volume                       True
user_persona                False
initial_package              True
is_transactional            False
is_marketing                False
is_behavioral               False
is_oem                      False
employee_count               True
risk                        False
geolocation_risk            False
geolocation_notes            True
name_risk                   False
name_notes    

In [3]:
yo = train[['banned_ip', 'banned_email','name_risk', 'risk', 'ip_count']]
yo.head()

,banned_ip,banned_email,name_risk,risk,ip_count
0,6,1,0,-100,8
1,5,1,0,-90,5
2,0,0,0,-44,1
3,4,2,0,-72,4
4,0,0,0,-34,1


In [4]:
def drop_features(df):
    return df.drop(['website','city','state','zip','country','registration_ip','company','created_at','multifactor_country_code','is_authy_verified','lead_source','marketing_channel','volume','user_persona','initial_package','is_transactional','is_marketing','is_behavioral','is_oem','employee_count','geolocation_risk','geolocation_notes','name_notes','ip_risk','ip_notes','community_risk','community_notes','fingerprint_risk','fingerprint_notes','email_risk','email_notes','activity_risk','activity_notes','ip_cluster','email_cluster','mfa_required','mfa_completed','whitelabel_required','whitelabel_completed','payment_required','payment_completed','profile_completed','email_completed'], axis=1)

train = drop_features(train)
test = drop_features(test)
train.head()

,id,label,banned_ip,ip_count,banned_email,risk,name_risk
0,0,1,6,8,1,-100,0
1,1,1,5,5,1,-90,0
2,2,0,0,1,0,-44,0
3,3,1,4,4,2,-72,0
4,4,0,0,1,0,-34,0


In [5]:
from sklearn import preprocessing
def encode_features(df_train, df_test):
    features = ['banned_ip', 'banned_email','name_risk', 'ip_count']
    df_combined = pd.concat([df_train[features], df_test[features]])
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        df_train[feature] = le.transform(df_train[feature])
        df_test[feature] = le.transform(df_test[feature])
    return df_train, df_test
    
data_train, data_test = encode_features(train, test)
data_train.head()

,id,label,banned_ip,ip_count,banned_email,risk,name_risk
0,0,1,6,7,1,-100,6
1,1,1,5,4,1,-90,6
2,2,0,0,0,0,-44,6
3,3,1,4,3,2,-72,6
4,4,0,0,0,0,-34,6


In [9]:
from sklearn.model_selection import train_test_split

X_all = data_train.drop(['label', 'id'], axis=1)
y_all = data_train['label']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=123)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
clf = RandomForestClassifier()

# Choose some parameter combinations to try
parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.942248375408


In [12]:
from sklearn.cross_validation import KFold

def run_kfold(clf):
    kf = KFold(891, n_folds=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_kfold(clf)

Fold 1 accuracy: 1.0
Fold 2 accuracy: 0.9438202247191011
Fold 3 accuracy: 0.9662921348314607
Fold 4 accuracy: 0.9662921348314607
Fold 5 accuracy: 0.9775280898876404
Fold 6 accuracy: 0.9101123595505618
Fold 7 accuracy: 0.9325842696629213
Fold 8 accuracy: 0.9775280898876404
Fold 9 accuracy: 1.0
Fold 10 accuracy: 0.9887640449438202
Mean Accuracy: 0.9662921348314608


C:\Users\matt\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
ids = data_test['id']
predictions = clf.predict(data_test.drop('id', axis=1))


output = pd.DataFrame({ 'id' : ids, 'label': predictions })
output.to_csv('yo.csv', index = False)
output.head()

,id,label
0,333927,0
1,333928,0
2,333929,0
3,333930,0
4,333931,0


In [ ]:
from IPython.display import Image
Image(filename='pic.png')